# PIP installs and Prelimiary setup

In [5]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

!pip install transformers
!pip install torch
!pip install scipy
from transformers import AutoTokenizer, AutoModel, TFAutoModel
import numpy as np
from scipy.spatial.distance import cosine
from collections import defaultdict    

# Emoji classification

The emoji classifier only uses 20 emojis for classifying.
The results are printed in rank order

In [15]:
MODEL = "cardiffnlp/twitter-roberta-base"


# Chosen Tasks:
# emoji, hate, sentiment

task='emoji'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping 
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PyTorch
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

text = "I love photographing American animals"
    # Others to try / with interesting results
    #text = "Christmas is the best time of year!"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)


ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) 🇺🇸 0.342
2) 😂 0.1345
3) 😍 0.1104
4) 😁 0.0741
5) 😊 0.0692
6) 😎 0.0416
7) ❤ 0.0341
8) 😉 0.0286
9) 💙 0.0286
10) 💜 0.0254
11) 😜 0.0222
12) 📷 0.0166
13) 📸 0.0144
14) 💕 0.0133
15) 😘 0.0127
16) 💯 0.0085
17) ✨ 0.008
18) 🔥 0.0073
19) 🎄 0.0068
20) ☀ 0.0018
